In [ ]:
filedata = open("/content/drive/MyDrive/WaterRegulationsProject/00000519c.txt","r").read()

In [ ]:
filedata[1:5000]

In [ ]:
import re
indices = [m.start() for m in re.finditer('\x0c', filedata)]
pages = []
index = 0
for end in indices:
  pages.append(filedata[index:end+1])
  index = end+1

In [ ]:
len(pages)

In [ ]:
import pandas as pd
df = pd.DataFrame(
    {'page_no': range(1, len(pages)+1),
     'original_content': pages,
    })
df.head()

In [ ]:
df.tail()

In [ ]:
print(df['original_content'].loc[83])

# Remove unwanted pages (Index, appendix etc)

In [ ]:
not_required_pages = []

# Index
not_required_pages.extend(range(1,7))
not_required_pages.extend([292, 293, 340, 395, 429, 430, 431, 432, 505, 506, 507, 508, 509, ])

# References and images
not_required_pages.extend(range(554, 587))

# Testing report form
not_required_pages.extend([117, 118, 119, 126, 127, 128, 129])

# Bibliography
not_required_pages.extend(range(131, 138))

# Water Body Classifications in State
not_required_pages.extend(range(180, 198))

# Allowance for Facilities Planning and Design
not_required_pages.extend([240, 241])

# Addresses
not_required_pages.extend([265, 266, 267])

# not_required_pages

In [ ]:
df = df[~df.page_no.isin(not_required_pages)].copy()

In [ ]:
page7_str = df.loc[6, 'original_content']
df.loc[6, 'original_content'] = page7_str[page7_str.index('Subchapter 1 Wastewater Permit Regulations'):]
df.loc[6, 'original_content']

In [ ]:
df.head()

In [ ]:
df.tail()

# Clean page breaks and page numbers from footer.

Clean bullet points  1., 2., 3.,.. (1), (2), (3),... (a), (b), (c), A., B., C.,

In [ ]:
df['clean_content'] = df.original_content.apply(lambda x: re.sub(r"[0-9]+\n\x0c", "", x).rstrip())

In [ ]:
print(df['clean_content'].loc[83])

             (i)     The Linear Partition Coefficient is calculated from Table 5-
                     4 in the Technical Guidance Manual using the following
                     formula:

                        K p = K po * TSS a

      Where: Kp    = Linear Partition Coefficient
      Kpo = Regression Coefficient (from table)
      TSS = Total Suspended Solids Concentration (site specific 15th
              percentile)
      a     = Exponent Constant (from table)

             (ii)    The Dissolved Fraction is then calculated as follows:

                    C            1
                      =
                    CT 1 + K p * TSS *10 −6

      Where: C/CT = Dissolved Fraction of metal

      (2)    Alternately, the permittee may determine the dissolved fraction
             directly by analyzing paired samples of site water for dissolved and
             total recoverable metals using EPA's "clean" analytical techniques
             and sampling procedures. This determination may

In [ ]:
p1 = re.compile(r"^[A-Z]\. (.*)$")
p2 = re.compile(r"^[0-9]+\. (.*)$")
p3 = re.compile(r"^\([a-zA-Z]\) (.*)$")
p4 = re.compile(r"^\([0-9]+\) (.*)$")
p5 = re.compile(r"^\((i|ii|iii|iv|v|vi|vii|viii|ix|x)\) (.*)$")
p6 = re.compile(r"(?i)^Appendix ?[A-Z] (.*)$")
p7 = re.compile(r"^[a-z]  (.*)$")

def clean_bullet(x):
  lines = x.split('\n')
  new_lines = []
  for line in lines:
    new_line = p1.sub(r"\1", line.strip()).strip()
    new_line = p2.sub(r"\1", new_line).strip()
    new_line = p3.sub(r"\1", new_line).strip()
    new_line = p4.sub(r"\1", new_line).strip()
    new_line = p5.sub(r"\2", new_line).strip()
    new_line = p6.sub(r"\1", new_line).strip()
    new_line = p7.sub(r"\1", new_line).strip()
    if new_line:
      new_lines.append(new_line)

  return '\n'.join(new_lines)

df['clean_content'] = df.clean_content.apply(clean_bullet)

In [ ]:
print(df['clean_content'].loc[99])

O'Conner-Dobbins equation may be used for streams with depths greater than 5 feet and
where there are adequate stream depth profiles or reasonable estimates available. If stream flow
is less than 280 cfs, Tsivoglou escape coefficient values should be considered.
12 .9 *V 0.5
Ka =
D1.5
where:
V      = velocity in ft./sec.
D      = depth in ft.
Ka     = reaeration rate 1/day (base e @ 20°C).
A minimum Ka value of 0.15/day will be used except in the case mentioned under Kd
where Ka/Kd is not less than 2.
In the WASP model COVAR may be considered.
Stream Background Conditions
Assume the following stream background conditions unless data show otherwise.
DO = 85% of saturation at assumed stream temperature (table attached)
CBODU = 2.0 mg/l
CBOD5 = 1.33 mg/l
NBODU = 0.5 mg/l
NH3-N = 0.10 mg/l
Photosynthesis / Respiration
Input values for P and R (mg/l/day) can be determined in stream studies using the:
Delta Method
Diurnal Curve Method
Light/Dark Bottle Method
In the absence of field data, P 

# Read chemical names


In [ ]:
for page in [172, 173]:
  print(df['original_content'].loc[page])
  print()

                                                          TABLE 2
                                             Numeric Criteria for All Waters (µg/l)


                                                         Fresh Water                Salt Water             Human Health
 CAS j               Parameter                    Acute         Chronic      Acute         Chronic     Organisms    Water &
                                                                                                          Only     Organisms
 309002    Aldrin                                  3.0                        1.3                       0.000050    0.000049
                                                     f                f         f                f
7664417    Ammonia
7440382    Arsenic (III), Total Dissolved         340 e           150 e        69                36
7440382    Arsenic, Total Dissolved                                                                       24h        0.078h
7440439    C

In [ ]:
chemical_dict={}
chemical_dict['Aldrin'] = ['Aldrin']
chemical_dict['Chlordane'] = ['Chlordane']
chemical_dict['DDT'] = ['Dichlorodiphenyltrichloroethane', 'C14H9Cl5']
chemical_dict['Dieldrin'] = ['Dieldrin']
chemical_dict['Dioxin'] = ['TCDD']
chemical_dict['Endrin'] = ['Endrin']
chemical_dict['Heptachlor'] = ['Heptachlor']
chemical_dict['Lindane'] = ['BHC']
chemical_dict['Phenol'] = ['C6H6O']
chemical_dict['Pentachlorophenol'] = ['Pentachlorophenol', 'PCB']
chemical_dict['Toxaphene'] = ['Toxaphene']
chemical_dict['CBOD'] = ['Carbonaceous biochemical oxygen demand', 'CBODU']
chemical_dict['NBOD'] = ['Nitrogenous biochemical oxygen demand', 'NBODU']
chemical_dict['BOD'] = ['Biochemical oxygen demand', 'BOD']

#Water Data CSV
chemical_dict['Nitrogen'] = ['N', 'TN', 'TKN']
chemical_dict['Ammonia'] = ['NH3']
chemical_dict['Nitrite'] = ['NO2']
chemical_dict['Nitrate'] = ['NO3']
chemical_dict['Phosphorus'] = ['P', 'TP']
chemical_dict['Oxygen'] = ['O', 'DO']
chemical_dict['ph'] = ['PH']
chemical_dict['Alkalinity'] = ['CACO3', 'BICARBONATE', 'CARBONATE']
chemical_dict['Arsenic'] = ['AS']
chemical_dict['Cadmium'] = ['Cd']
chemical_dict['Carbon'] = ['C', 'TOC']
chemical_dict['Chlorine'] = ['Cl', 'Chloride']
chemical_dict['Chlorophyll'] = ['CHLA', 'CHLAC']
chemical_dict['Chromium'] = ['CR']
chemical_dict['Copper'] = ['CU']
chemical_dict['Fluoride'] = ['F']
chemical_dict['Iron'] = ['FE']
chemical_dict['Lead'] = ['PB']
chemical_dict['Magnesium'] = ['MG']
chemical_dict['Nickel'] = ['NI']
chemical_dict['Pheophytin'] = ['PHEO']
chemical_dict['Potassium'] = ['K']
chemical_dict['Silver'] = ['AG']
chemical_dict['Sodium'] = ['NA']
chemical_dict['Sulphate'] = ['SO4', 'SULFUR', 'Sulfate']
chemical_dict['Zinc'] = ['ZN']
chemical_dict['Salinity'] = ['SALINITY']
chemical_dict['Aluminium'] = ['AL', 'Aluminium', 'Aluminum']
chemical_dict['Antimony'] = ['SB']
chemical_dict['Barium'] = ['BA']
chemical_dict['Boron'] = ['B']
chemical_dict['Diuron'] = ['DI']
chemical_dict['Linuron'] = ['LINURON']
chemical_dict['Mecoprop'] = ['BA']
chemical_dict['Glyphosate'] = ['GLYPHOSATE']
chemical_dict['Selenium'] = ['SE']
chemical_dict['Sucralose'] = ['SU']
chemical_dict['Thallium'] = ['TL']
chemical_dict['Manganese'] = ['MN']
chemical_dict['Calcium'] = ['CA']
chemical_dict['Silica'] = ['SI']

#FromWHO Book 4th Edition
chemical_dict['Asbestos'] = ['Chrysotile']
chemical_dict['Bentazone'] = ['Bentazone']
chemical_dict['Beryllium'] = ['Beryllium', 'BE']
chemical_dict['Bromide'] = ['BR']
chemical_dict['Bromochloroacetate'] = ['Bromochloroacetate']
chemical_dict['Carbaryl'] = ['Carbaryl']
chemical_dict['Chloralhydrate'] = ['Chloral hydrate']
chemical_dict['Chlorine dioxide'] = ['clo2']
chemical_dict['Chloroacetone'] = ['Chloroacetone']
chemical_dict['Chloropicrin'] = ['Chloropicrin']
chemical_dict['Cyanide'] = ['CN',]
chemical_dict['Cyanogen chloride'] = ['Cyanogen chloride']
chemical_dict['Dibromoacetate'] = ['Dibromoacetate']
chemical_dict['Dichloramine'] = ['NHCl2']
chemical_dict['Dichlorobenzene'] = ['C6H4Cl2']
chemical_dict['Dichloroethane'] = ['C2H4Cl2']
chemical_dict['Dichloroethene'] = ['C2H2Cl2']
chemical_dict['Dichlorophenol'] = ['C6H4Cl2O']
chemical_dict['Dichloropropane'] = ['C3H6Cl2']
chemical_dict['Dicofol'] = ['Dicofol']
chemical_dict['Diflubenzuron'] = ['Diflubenzuron']
chemical_dict['Endosulfan'] = ['Endosulfan']
chemical_dict['Fenitrothion'] = ['Fenitrothion']
chemical_dict['Fluoranthene'] = ['Fluoranthene']
chemical_dict['Formaldehyde'] = ['Formaldehyde']
chemical_dict['Hexachlorobenzene'] = ['C6Cl6']
chemical_dict['Hydrogen sulfide'] = ['H2S']
chemical_dict['Iodine'] = ['I']
chemical_dict['Malathion'] = ['Malathion']
chemical_dict['MCPA'] = ['MCPA']
chemical_dict['Molybdenum'] = ['Mo']
chemical_dict['Nitrobenzene'] = ['Nitrobenzene']
chemical_dict['Phenylphenol'] = ['C12H10O']
chemical_dict['Trichloramine'] = ['NCl3']
chemical_dict['Acrylamide'] = ['C3H5NO']
chemical_dict['benzene'] = ['C6H6']
chemical_dict['Bromate'] = ['bro3']
chemical_dict['Bromodichloromethane'] = ['MCPA']
chemical_dict['Bromoform'] = ['CHBr3']
chemical_dict['Carbon tetrachloride'] = ['CCL4']
chemical_dict['Chlorate'] = ['CLO3']
chemical_dict['Chlorite'] = ['CLO2']
chemical_dict['Chloroform'] = ['CHCl3']
chemical_dict['Cyanazine'] = ['Cyanazine']
chemical_dict['Dichloroacetate'] = ['Dichloroacetate']
chemical_dict['Dibromoethane'] = ['Dibromoethane']
chemical_dict['Dichloromethane'] = ['CH2Cl2']
chemical_dict['Dimethoate'] = ['Dimethoate']
chemical_dict['Dioxane'] = ['Dioxane']
chemical_dict['Ethylbenzene'] = ['C8H10']
chemical_dict['Hydroxyatrazine'] = ['Hydroxyatrazine']
chemical_dict['Isoproturon'] = ['Isoproturon']
chemical_dict['Lindane'] = ['c6h6cl6']
chemical_dict['Mercury'] = ['Hg']
chemical_dict['Metolachlor'] = ['Metolachlor']
chemical_dict['Molinate'] = ['Molinate']
chemical_dict['Monochloramine'] = ['NH2Cl']
chemical_dict['Monochloroacetate'] = ['Monochloroacetate']
chemical_dict['Cyanuric acid'] = ['Sodium dichloroisocyanurate', 'C3H3N3O3']
chemical_dict['Styrene'] = ['C8H8']
chemical_dict['Tetrachloroethene'] = ['C2Cl4']
chemical_dict['Toluene'] = ['C7H8']
chemical_dict['Trichloroacetate'] = ['Trichloroacetate']
chemical_dict['Trichloroethene'] = ['Trichloroethene']
chemical_dict['Trichlorophenol'] = ['Trichlorophenol']
chemical_dict['Trifluralin'] = ['Trifluralin']
chemical_dict['Trihalomethanes'] = ['THM']
chemical_dict['Uranium'] = ['U']
chemical_dict['Vinyl chloride'] = ['C2H3Cl']
chemical_dict['Xylene'] = ['Xylene', 'Xylenes', 'C8H10']

In [ ]:
from collections import OrderedDict
def get_counts(x, full_only=False):
  x = x.upper()
  d = OrderedDict()

  for chemical, vals in chemical_dict.items():
    if full_only:
      counts = len(re.findall(r'\b'+chemical.upper()+r'\b', x))
    else:
      counts = len(re.findall(r'\b('+chemical.upper()+'|'+'|'.join([val.upper() for val in vals])+r')\b', x))
    d[chemical] = counts

  return pd.Series(d)

counts_df_full_only = df.clean_content.apply(get_counts, full_only=True)
counts_df_all = df.clean_content.apply(get_counts, full_only=False)

In [ ]:
counts_df_full_only['Ammonia'].head()

0    0
1    0
2    0
3    0
4    1
Name: Ammonia, dtype: int64

In [ ]:
df.loc[counts_df_full_only[counts_df_full_only.Ammonia>0].index]

,page_no,original_content,clean_content
4,5,Rule 1.2.7 Bibliography\n\nExhibit A to Chapte...,Rule 1.2.7 Bibliography\nExhibit A to Chapter ...
76,77,(4) some oxygen demanding wastewate...,"some oxygen demanding wastewaters, other than ..."
79,80,(2) Alternative methods may be util...,Alternative methods may be utilized to determi...
102,103,...,Acute Chronic\nFresh 19 1...
103,104,Final ammonia allocations will be reported as ...,Final ammonia allocations will be reported as ...
165,166,toxicity will be determined in ac...,toxicity will be determined in accordance with...
171,172,TABLE...,TABLE 2 Notes\nThe CMC = 1/[(f1/CMC1) + (f2/CM...
172,173,...,TABLE 2\nNumeric Criteria for All Waters (µg/l...
177,178,E. EPHEMERAL STREAM CLASSIFICATION:\n\n ...,EPHEMERAL STREAM CLASSIFICATION:\nWaters in th...
481,482,Ephemeral ...,Ephemeral ...


In [ ]:
df.loc[counts_df_all[counts_df_all.Ammonia>0].index]

,page_no,original_content,clean_content
4,5,Rule 1.2.7 Bibliography\n\nExhibit A to Chapte...,Rule 1.2.7 Bibliography\nExhibit A to Chapter ...
76,77,(4) some oxygen demanding wastewate...,"some oxygen demanding wastewaters, other than ..."
79,80,(2) Alternative methods may be util...,Alternative methods may be utilized to determi...
99,100,3. O'Conner-Dobbins equation may be used...,O'Conner-Dobbins equation may be used for stre...
100,101,"1. In the STREAM Model, sediment oxygen d...","In the STREAM Model, sediment oxygen demand (""..."
102,103,...,Acute Chronic\nFresh 19 1...
103,104,Final ammonia allocations will be reported as ...,Final ammonia allocations will be reported as ...
165,166,toxicity will be determined in ac...,toxicity will be determined in accordance with...
171,172,TABLE...,TABLE 2 Notes\nThe CMC = 1/[(f1/CMC1) + (f2/CM...
172,173,...,TABLE 2\nNumeric Criteria for All Waters (µg/l...


In [ ]:
print(df.clean_content.loc[99])

O'Conner-Dobbins equation may be used for streams with depths greater than 5 feet and
where there are adequate stream depth profiles or reasonable estimates available. If stream flow
is less than 280 cfs, Tsivoglou escape coefficient values should be considered.
12 .9 *V 0.5
Ka =
D1.5
where:
V      = velocity in ft./sec.
D      = depth in ft.
Ka     = reaeration rate 1/day (base e @ 20°C).
A minimum Ka value of 0.15/day will be used except in the case mentioned under Kd
where Ka/Kd is not less than 2.
In the WASP model COVAR may be considered.
Stream Background Conditions
Assume the following stream background conditions unless data show otherwise.
DO = 85% of saturation at assumed stream temperature (table attached)
CBODU = 2.0 mg/l
CBOD5 = 1.33 mg/l
NBODU = 0.5 mg/l
NH3-N = 0.10 mg/l
Photosynthesis / Respiration
Input values for P and R (mg/l/day) can be determined in stream studies using the:
Delta Method
Diurnal Curve Method
Light/Dark Bottle Method
In the absence of field data, P 

In [ ]:
counts_df_all.sum(axis=1).sort_values(ascending=False)

172    38
164    33
483    31
100    31
457    29
       ..
344     0
339     0
318     0
312     0
585     0
Length: 586, dtype: int64

In [ ]:
page_param_count = counts_df_full_only.sum(axis=1).sort_values(ascending=False).to_frame()
page_param_count[page_param_count[0]>0].shape

(66, 1)

In [ ]:
counts_df_all.loc[172]

Aldrin             1
Chlordane          1
DDT                1
Dieldrin           1
Dioxin             2
                  ..
Trifluralin        0
Trihalomethanes    0
Uranium            0
Vinyl chloride     0
Xylene             0
Name: 172, Length: 124, dtype: int64

In [ ]:
print(df.clean_content.loc[171])

TABLE 2 Notes
The CMC = 1/[(f1/CMC1) + (f2/CMC2)] where f1 and f2 are the fractions of total selenium
that are treated as selenite and selenate, respectively, and CMC1 and CMC2 are 185.9 µg/l
and 12.83 µg/l. The value in the table is calculated assuming a worst case scenario in
which all selenium is present as selenate.
Hardness dependent parameter. Criteria are indicated at hardness of 50 mg/l as CaCO3.
Equations for criteria calculation of hardness dependent parameters can be found in Quality
Criteria for Water. The equation is applicable for instream hardness ranges from 25 mg/l
to 400 mg/l. If instream hardness is less than 25 mg/l, then a hardness value of 25 mg/l
should be used to calculate the criteria. If instream hardness is greater than 400 mg/l, then
a hardness of 400 mg/l should be used to calculate the criteria.
Criteria for pentachlorophenol are based on a pH dependent equation as found in Quality
Criteria for Water. Values listed are for a pH of 7.0 s.u.
Site specific cr